In [63]:
import json
import itertools
import os
import pandas as pd
from matplotlib import pyplot as plt
from IPython.core.display import HTML

In [44]:
DATA_PATH = "../SOSD/data/osm_cellids_200M_uint64"

In [45]:
top_only_layers = ["radix", "bradix"]
anywhere_layers = ["linear", "cubic"]
specialty_top_layers = ["histogram", "loglinear", "normal", "lognormal"]
branching_factors = [2**x for x in range(7, 20)]

In [46]:
all_top_layers = top_only_layers + anywhere_layers

In [47]:
# first, build a grid of the most likely configs
configs = []
for top in all_top_layers:
    for bot in anywhere_layers:
        for bf in branching_factors[::2]:
            configs.append({"layers": f"{top},{bot}", "branching factor": bf})
            
# next, build a few tests to see if a speciality layer would help
for top in specialty_top_layers:
    if top == "histogram":
        for bot in anywhere_layers:
            for bf in [64, 128, 256]:
                configs.append({"layers": f"{top},{bot}", "branching factor": bf})       
    else:
        # not a histogram
        for bot in anywhere_layers:
            for bf in branching_factors[::3]:
                configs.append({"layers": f"{top},{bot}", "branching factor": bf})       



In [48]:
print("Testing", len(configs), "initial configurations.")
with open("step1.json", "w") as f:
    json.dump({"configs": configs}, f)
    

Testing 78 initial configurations.


In [50]:
cmd = f"RUST_BACKTRACE=1 RUST_LOG=trace cargo run --release {DATA_PATH} --param-grid step1.json"
os.system(cmd)

0

In [53]:
with open("results.json", "r") as f:
    step1_results = json.load(f)

step1_results = pd.DataFrame(step1_results)

In [65]:
display(HTML(step1_results.to_html()))

,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
0,"radix,linear",128,1.677722e+07,24.181279,48362559,24.181279,17536,16512
1,"radix,linear",512,2.915224e+06,4.406203,8812405,4.406203,69760,65664
2,"radix,linear",2048,1.248769e+06,5.211855,10423711,5.211855,278656,262272
3,"radix,linear",8192,2.677103e+05,2.301554,4603109,2.301554,1114240,1048704
4,"radix,linear",32768,1.131330e+05,1.234042,2468084,1.234042,4456576,4194432
5,"radix,linear",131072,4.993954e+04,0.989075,1978150,0.989075,17825920,16777344
6,"radix,cubic",128,8.388608e+06,14.061114,28122228,14.061114,33920,32896
7,"radix,cubic",512,2.336881e+06,3.883072,7766143,3.883072,135296,131200
8,"radix,cubic",2048,1.048576e+06,3.054265,6108530,3.054265,540800,524416
9,"radix,cubic",8192,3.648162e+05,1.783331,3566662,1.783331,2162816,2097280


In [121]:
def pareto_mask(df):
    # find Pareto efficient RMIs
    mask = []
    for idx1, el1 in df.iterrows():
        my_size = el1["size linear search"]
        my_error = el1["max error"]
        for idx2, el2 in df.iterrows():
            if idx1 == idx2:
                continue

            if (el2["size linear search"] <= my_size) and (el2["max error"] <= my_error):
                mask.append(False)
                break
        else:
            mask.append(True)
    return mask

In [122]:
mask = pareto_mask(step1_results)
pareto = step1_results[mask]
print("Found", len(pareto), "Pareto efficient models")
display(HTML(pareto.to_html()))

Found 11 Pareto efficient models


,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
24,"linear,linear",128,2.339913e+06,5.746346,11492692,5.746346,17536,16512
25,"linear,linear",512,7.084016e+05,1.965939,3931879,1.965939,69760,65664
56,"loglinear,linear",8192,2.621440e+05,0.757990,1515980,0.757990,1114240,1048704
61,"loglinear,cubic",65536,7.002274e+04,0.224099,448197,0.224099,17301632,16777344
62,"normal,linear",128,3.185062e+06,3.375293,6750586,3.375293,17600,16576
64,"normal,linear",8192,1.633491e+05,0.475484,950969,0.475484,1114304,1048768
65,"normal,linear",65536,4.135816e+04,0.308672,617344,0.308672,8913088,8388800
67,"normal,cubic",1024,7.341116e+05,0.969581,1939162,0.969581,270528,262336
68,"normal,cubic",8192,1.469672e+05,0.423521,847043,0.423521,2162880,2097344
69,"normal,cubic",65536,3.340031e+04,0.133224,266449,0.133224,17301696,16777408


In [123]:
candidate_layers = set(pareto["layers"])
next_configs = []
for candidate in candidate_layers:
    if candidate.startswith("histogram"):
        for bf in [32, 300, 512]:
            next_configs.append({"layers": candidate, "branching factor": bf})       

    else:
        already_known = step1_results[step1_results.layers == candidate]["branching factor"].to_list()
        for bf in sorted(set(branching_factors) - set(already_known)):
            next_configs.append({"layers": candidate, "branching factor": bf})

In [124]:
            
print("Testing", len(next_configs), "additional configurations.")
with open("step2.json", "w") as f:
    json.dump({"configs": next_configs}, f)
    

Testing 46 additional configurations.


In [125]:
cmd = f"RUST_BACKTRACE=1 RUST_LOG=trace cargo run --release {DATA_PATH} --param-grid step2.json"
os.system(cmd)

0

In [132]:
with open("results.json", "r") as f:
    step2_results = json.load(f)

step2_results = pd.DataFrame(step2_results)
all_results = pd.concat((step1_results, step2_results)).reset_index(drop=True)
mask = pareto_mask(all_results)
all_results.sort_values("max error")

,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
100,"normal,cubic",131072,2.251186e+04,0.116613,233227,0.116613,34603200,33554624
69,"normal,cubic",65536,3.340031e+04,0.133224,266449,0.133224,17301696,16777408
101,"normal,cubic",262144,1.314978e+04,0.136809,273618,0.136809,69206208,67109056
47,"cubic,cubic",131072,2.479151e+04,0.139613,279226,0.139613,34603264,33554688
116,"loglinear,cubic",131072,3.889055e+04,0.144930,289860,0.144930,34603136,33554560
...,...,...,...,...,...,...,...,...
102,"loglinear,linear",256,3.204531e+06,11.464436,22928871,11.464436,34944,32896
54,"loglinear,linear",128,4.194304e+06,11.555172,23110345,11.555172,17536,16512
110,"loglinear,cubic",256,3.001777e+06,12.528464,25056927,12.528464,67712,65664
6,"radix,cubic",128,8.388608e+06,14.061114,28122228,14.061114,33920,32896


In [133]:
display(HTML(all_results[mask].sort_values("max error").to_html()))

,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
100,"normal,cubic",131072,2.251186e+04,0.116613,233227,0.116613,34603200,33554624
69,"normal,cubic",65536,3.340031e+04,0.133224,266449,0.133224,17301696,16777408
61,"loglinear,cubic",65536,7.002274e+04,0.224099,448197,0.224099,17301632,16777344
115,"loglinear,cubic",32768,8.936814e+04,0.245487,490975,0.245487,8650880,8388736
68,"normal,cubic",8192,1.469672e+05,0.423521,847043,0.423521,2162880,2097344
64,"normal,linear",8192,1.633491e+05,0.475484,950969,0.475484,1114304,1048768
56,"loglinear,linear",8192,2.621440e+05,0.757990,1515980,0.757990,1114240,1048704
96,"normal,cubic",2048,3.824983e+05,0.820843,1641686,0.820843,540864,524480
67,"normal,cubic",1024,7.341116e+05,0.969581,1939162,0.969581,270528,262336
71,"lognormal,linear",1024,7.412268e+05,1.351709,2703418,1.351709,139456,131264


In [134]:
def human_size(bytes, units=[' bytes','KB','MB','GB','TB', 'PB', 'EB']):
    """ Returns a human readable string reprentation of bytes"""
    return str(bytes) + units[0] if bytes < 1024 else human_size(bytes>>10, units[1:])

all_results["size linear search"] = [human_size(x) for x in all_results["size linear search"].to_list()]
all_results["size binary search"] = [human_size(x) for x in all_results["size binary search"].to_list()]
all_results[mask].sort_values("max error")

,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
100,"normal,cubic",131072,2.251186e+04,0.116613,233227,0.116613,33MB,32MB
69,"normal,cubic",65536,3.340031e+04,0.133224,266449,0.133224,16MB,16MB
61,"loglinear,cubic",65536,7.002274e+04,0.224099,448197,0.224099,16MB,16MB
115,"loglinear,cubic",32768,8.936814e+04,0.245487,490975,0.245487,8MB,8MB
68,"normal,cubic",8192,1.469672e+05,0.423521,847043,0.423521,2MB,2MB
64,"normal,linear",8192,1.633491e+05,0.475484,950969,0.475484,1MB,1MB
56,"loglinear,linear",8192,2.621440e+05,0.757990,1515980,0.757990,1MB,1MB
96,"normal,cubic",2048,3.824983e+05,0.820843,1641686,0.820843,528KB,512KB
67,"normal,cubic",1024,7.341116e+05,0.969581,1939162,0.969581,264KB,256KB
71,"lognormal,linear",1024,7.412268e+05,1.351709,2703418,1.351709,136KB,128KB


In [130]:
all_results

,layers,branching factor,average error,average error %,max error,max error %,size binary search,size linear search
0,"radix,linear",128,1.677722e+07,24.181279,48362559,24.181279,17536,16KB
1,"radix,linear",512,2.915224e+06,4.406203,8812405,4.406203,69760,64KB
2,"radix,linear",2048,1.248769e+06,5.211855,10423711,5.211855,278656,256KB
3,"radix,linear",8192,2.677103e+05,2.301554,4603109,2.301554,1114240,1MB
4,"radix,linear",32768,1.131330e+05,1.234042,2468084,1.234042,4456576,4MB
...,...,...,...,...,...,...,...,...
119,"linear,linear",1024,5.411146e+05,3.355415,6710829,3.355415,139392,128KB
120,"linear,linear",4096,2.306924e+05,2.011261,4022523,2.011261,557184,512KB
121,"linear,linear",16384,1.127650e+05,0.655880,1311761,0.655880,2228352,2MB
122,"linear,linear",65536,5.819910e+04,0.655846,1311693,0.655846,8913024,8MB
